In [1]:
from collections import OrderedDict
import os
from tqdm import tqdm, trange
from IPython.display import clear_output
import time
import torch.nn as nn
import pandas as pd
import torch
from models.ae import MNISTAutoencoder
from models.flow import Flow
from models.generator import MNISTGenerator
import matplotlib.pyplot as plt
from dataset import MNISTDataset, MNIST_mean, MNIST_std, MNISTFlowDataset
from utils.run_manager import RunBuilder

if torch.cuda.is_available():
    devices = ['cuda']
else:
    devices = ['cpu']
print('starting')


params = OrderedDict(
    lr = [0.001],
    batch_size = [32],
    device = devices,
    shuffle = [True],
    num_workers = [5],
    beta = [10],
    z_dim = [2], 
    manual_seed = [1265],
    loss_func = [nn.MSELoss]
)


ae = torch.load('trained_models/mnist_ae_noact.model')

train_set = MNISTFlowDataset(path='data/MNIST/latent')

run_count = 0
models = []

run_data = []

data_load_time = 0
forward_time = 0
for run in RunBuilder.get_runs(params):
  
    run_count += 1
    device = torch.device(run.device)
    
    model = Flow()
    model = model.to(device)

    loader = torch.utils.data.DataLoader(train_set, 
                                         batch_size=run.batch_size, 
                                         shuffle=run.shuffle, 
                                         num_workers=run.num_workers
                                        )
    optimizer = torch.optim.Adam(model.parameters(), lr=run.lr)
    num_batches = len(train_set)/run.batch_size

    for epoch in range(100):
        total_recons_loss = 0
        batch_count = 0
        
        results = OrderedDict()
        results['run_count'] = run_count
        results['epoch'] = epoch
        results['data_fidelity'] = total_recons_loss/num_batches
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        results['device'] = run.device
        results['z_dim'] = run.z_dim
        
#         run_data.append(results)
        df2 = pd.DataFrame.from_dict(run_data, orient='columns')
        
        
        for batch in tqdm(loader):
            batch_count +=1
            optimizer.zero_grad()

            X = batch
            X = X.to(device=run.device)
            loss = model.loss(X)
            
            loss.backward()
            optimizer.step()
            print(loss)
            
            total_recons_loss += loss.item()
            
            
        results = OrderedDict()
        results['run_count'] = run_count
        results['epoch'] = epoch
        results['data_fidelity'] = total_recons_loss/num_batches
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        results['device'] = run.device
        results['z_dim'] = run.z_dim
        
        run_data.append(results)
        df2 = pd.DataFrame.from_dict(run_data, orient='columns')
        clear_output(wait=True)
        display(df2)
            
        torch.save(model, 'trained_models/mnist_flow_4_test.model'.format(run.lr,run.beta, run.z_dim))
    models.append(model)

,run_count,epoch,data_fidelity,batch_size,lr,device,z_dim
0,1,0,16.876468,32,0.001,cuda,2


  0%|          | 8/1875 [00:00<01:11, 26.28it/s]

tensor(8.1964, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.2564, device='cuda:0', grad_fn=<NegBackward>)
tensor(6.5899, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.7277, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.2776, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.2131, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.3671, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.3328, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.0819, device='cuda:0', grad_fn=<NegBackward>)
tensor(6.9236, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6015, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.9022, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8349, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8127, device='cuda:0', grad_fn=<NegBackward>)


  1%|▏         | 24/1875 [00:00<00:34, 54.30it/s]

tensor(8.0585, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.2047, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.7724, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6091, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.9247, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.2974, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.9581, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.5352, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6620, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8237, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.3587, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.2665, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6789, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.0348, device='cuda:0', grad_fn=<NegBackward>)


...

tensor(7.9389, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.3995, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.9208, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6599, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6272, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.4472, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.2910, device='cuda:0', grad_fn=<NegBackward>)
tensor(6.9647, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.0609, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8360, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8336, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.7873, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.6694, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.5662, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.3918, device='cuda:0', grad_fn=<NegBackward>)


 37%|███▋      | 703/1875 [00:10<00:17, 67.78it/s]

tensor(7.1987, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.2013, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.1765, device='cuda:0', grad_fn=<NegBackward>)
tensor(8.0849, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.5305, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.9559, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.5586, device='cuda:0', grad_fn=<NegBackward>)
tensor(7.8054, device='cuda:0', grad_fn=<NegBackward>)


 38%|███▊      | 708/1875 [00:10<00:17, 68.45it/s]


KeyboardInterrupt: 

## Train Semi Supervised Flow

In [ ]:
from collections import OrderedDict
import os
from tqdm import tqdm, trange
from IPython.display import clear_output
import time
import torch.nn as nn
import pandas as pd
import torch
from models.ae import MNISTAutoencoder
from models.flow import FlowSemiSupervised
from models.generator import MNISTGenerator
import matplotlib.pyplot as plt
from dataset import MNISTDataset, MNIST_mean, MNIST_std, MNISTFlowSupervisedDataset
from utils.run_manager import RunBuilder

if torch.cuda.is_available():
    devices = ['cuda']
else:
    devices = ['cpu']
print('starting')


params = OrderedDict(
    lr = [0.001],
    batch_size = [5],
    device = devices,
    shuffle = [True],
    num_workers = [5],
    beta = [10],
    z_dim = [2], 
    manual_seed = [1265],
    loss_func = [nn.MSELoss]
)


ae = torch.load('trained_models/mnist_ae_noact.model')

train_set = MNISTFlowSupervisedDataset(path='data/MNIST/latent')

run_count = 0
models = []

run_data = []

data_load_time = 0
forward_time = 0
for run in RunBuilder.get_runs(params):
  
    run_count += 1
    device = torch.device(run.device)
    
    model = FlowSemiSupervised()
    model = model.to(device)

    loader = torch.utils.data.DataLoader(train_set, 
                                         batch_size=run.batch_size, 
                                         shuffle=run.shuffle, 
                                         num_workers=run.num_workers
                                        )
    optimizer = torch.optim.Adam(model.parameters(), lr=run.lr)
    num_batches = len(train_set)/run.batch_size

    for epoch in range(100):
        total_recons_loss = 0
        batch_count = 0
        
        results = OrderedDict()
        results['run_count'] = run_count
        results['epoch'] = epoch
        results['data_fidelity'] = total_recons_loss/num_batches
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        results['device'] = run.device
        results['z_dim'] = run.z_dim
        
#         run_data.append(results)
        df2 = pd.DataFrame.from_dict(run_data, orient='columns')
        
        
        for batch in tqdm(loader):
            batch_count +=1
            optimizer.zero_grad()

            X, labels = batch
            X = X.to(device=run.device)
            loss = model.loss(X, labels)
            
            loss.backward()
            optimizer.step()
            
#             print('loss', loss.item())
            total_recons_loss += loss.item()
            
            
        results = OrderedDict()
        results['run_count'] = run_count
        results['epoch'] = epoch
        results['data_fidelity'] = total_recons_loss/num_batches
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        results['device'] = run.device
        results['z_dim'] = run.z_dim
        
        run_data.append(results)
        df2 = pd.DataFrame.from_dict(run_data, orient='columns')
        clear_output(wait=True)
        display(df2)
            
        torch.save(model, 'trained_models/mnist_flow_semisupervised_1.model')
    models.append(model)

,run_count,epoch,data_fidelity,batch_size,lr,device,z_dim
0,1,0,90.553761,5,0.001,cuda,2
1,1,1,72.175842,5,0.001,cuda,2
2,1,2,68.334987,5,0.001,cuda,2
3,1,3,67.877819,5,0.001,cuda,2
4,1,4,68.088346,5,0.001,cuda,2
...,...,...,...,...,...,...,...
85,1,85,95.020182,5,0.001,cuda,2
86,1,86,79.650522,5,0.001,cuda,2
87,1,87,75.742274,5,0.001,cuda,2
88,1,88,73.581813,5,0.001,cuda,2


 44%|████▍     | 5337/12000 [01:22<01:40, 66.37it/s]

In [2]:
multi = torch.distributions.MultivariateNormal(torch.zeros(2).to(device), torch.eye(2).to(device))
single = torch.distributions.Normal(torch.tensor(0.).to(device), torch.tensor(1.).to(device))

In [20]:
a = torch.randn((2))

multi.log_prob(a.to(device))


tensor(-3.2666, device='cuda:0')

In [21]:
single.log_prob(a.to(device)).sum()

tensor(-3.2666, device='cuda:0')

In [5]:
model.nvp.priors[2].mean

tensor([2.1631, 6.6574], device='cuda:0')

In [23]:
prior = torch.distributions.Normal(torch.tensor(0.).to(device), torch.tensor(1.).to(device))

model.nvp.priors[0].log_prob(torch.randn((1)).to(device))

tensor([-24.8367,  -0.9224], device='cuda:0', dtype=torch.float64)

In [3]:
m = torch.distributions.MultivariateNormal(torch.zeros(2).to(device), torch.eye(2).to(device))

In [8]:
m.log_prob(torch.randn((2), dtype=torch.float16).to(device))

tensor(-4.2778, device='cuda:0')

In [3]:
from dataset import GMM
gmm = GMM()

In [6]:
type(gmm.means[0])

list

In [3]:
torch.zeros(2)

tensor([0., 0.])

## Save Latent representation

In [5]:
import torch
from dataset import MNISTDataset
ae = torch.load('trained_models/mnist_ae_noact.model')

ae.eval()
ae = ae.to('cpu')

ae_set = MNISTDataset(path='data/MNIST/processed',train=False, normalize=True)

latent = ae.encoder(ae_set.dataset.unsqueeze(dim=1))
lat = latent.detach().clone()

torch.save((lat, ae_set.labels), 'data/MNIST/latent/test.pt')

In [5]:
import torch
a = torch.tensor([0,1,2])

In [6]:
for i in a:
    print(a[i])

tensor(0)
tensor(1)
tensor(2)
